# Learn about Iceberg table "Hidden Partitions".

In [1]:
import import_ipynb
%run  /Users/satyak/iceberg/demo/Configuration.ipynb

:: loading settings :: url = jar:file:/Users/satyak/.pyenv/versions/3.9.1/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/satyak/.ivy2/cache
The jars for the packages stored in: /Users/satyak/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.3_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-98ceea84-7081-44e5-bb05-3e0b5e2417d7;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.5.0 in central
:: resolution report :: resolve 135ms :: artifacts dl 8ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.5.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.

24/07/19 09:02:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Vector(spark://localhost:61532/jars/org.apache.iceberg_iceberg-spark-runtime-3.3_2.12-1.5.0.jar)


In [2]:
prod_db_nm="db2"

In [3]:
spark.sql(f"""drop database if EXISTS spark_catalog.db2 cascade""")

DataFrame[]

In [4]:
spark.sql(f"""CREATE database spark_catalog."""+prod_db_nm)

DataFrame[]

### Create Hive Table

In [5]:
spark.sql(f"""drop TABLE IF  EXISTS spark_catalog."""+prod_db_nm+""".order_hive""" )

DataFrame[]

In [6]:
spark.sql(f"""CREATE external TABLE IF NOT EXISTS spark_catalog."""+prod_db_nm+""".order_hive ( 
order_id BIGINT, 
customer_id BIGINT, 
order_amount DECIMAL(10, 2), 
Order_ts  timestamp,
order_dt date,
order_hr int,
st string)
USING orc PARTITIONED BY (order_dt, order_hr, st) LOCATION '/tmp/ozone1/warehouse/db2/order_hive' """)

DataFrame[]

In [7]:
spark.sql("insert into spark_catalog.db2.order_hive values(1, 3,  19900.76,timestamp '2023-12-25T10:31:40', date '2023-12-25', 10, 'IL')")
spark.sql("insert into spark_catalog.db2.order_hive values(2, 39, 19900.76,timestamp '2023-12-25T10:31:40', date '2023-12-25', 10, 'IL')")
spark.sql("insert into spark_catalog.db2.order_hive values(3, 31, 19900.76,timestamp '2023-12-25T10:31:40', date '2023-12-25', 11, 'IL')")
spark.sql("insert into spark_catalog.db2.order_hive values(4, 32, 19900.76,timestamp '2023-12-25T10:31:40', date '2023-12-25', 12, 'IL')")
spark.sql("insert into spark_catalog.db2.order_hive values(5, 33, 19900.76,timestamp '2023-12-25T10:31:40', date '2023-12-25', 13, 'IL')")
spark.sql("insert into spark_catalog.db2.order_hive values(6, 33, 19900.76,timestamp '2023-12-25T10:31:40', date '2023-12-25', 14, 'IL')")
spark.sql("insert into spark_catalog.db2.order_hive values(7, 34, 19900.76,timestamp '2023-12-25T10:31:40', date '2023-12-25', 14, 'IL')")
spark.sql("insert into spark_catalog.db2.order_hive values(8, 35, 19900.76,timestamp '2023-12-25T10:31:40', date '2023-12-25', 15, 'IL')")
spark.sql("insert into spark_catalog.db2.order_hive values(9, 36, 19900.76,timestamp '2023-12-25T10:31:40', date '2023-12-25', 15, 'IL')")
spark.sql("insert into spark_catalog.db2.order_hive values(10,37, 19900.76,timestamp '2023-12-25T10:31:40', date '2023-12-25', 15, 'IL')")
spark.sql("insert into spark_catalog.db2.order_hive values(11,38, 19900.76,timestamp '2023-12-25T10:31:40', date '2023-12-25', 16, 'IL')")
          

DataFrame[]

In [8]:

spark.sql("select * from spark_catalog."+prod_db_nm+".order_hive ").show(100,False)

+--------+-----------+------------+-------------------+----------+--------+---+
|order_id|customer_id|order_amount|Order_ts           |order_dt  |order_hr|st |
+--------+-----------+------------+-------------------+----------+--------+---+
|10      |37         |19900.76    |2023-12-25 10:31:40|2023-12-25|15      |IL |
|10      |37         |19900.76    |2023-12-25 10:31:40|2023-12-25|15      |IL |
|3       |31         |19900.76    |2023-12-25 10:31:40|2023-12-25|11      |IL |
|3       |31         |19900.76    |2023-12-25 10:31:40|2023-12-25|11      |IL |
|7       |34         |19900.76    |2023-12-25 10:31:40|2023-12-25|14      |IL |
|7       |34         |19900.76    |2023-12-25 10:31:40|2023-12-25|14      |IL |
|11      |38         |19900.76    |2023-12-25 10:31:40|2023-12-25|16      |IL |
|11      |38         |19900.76    |2023-12-25 10:31:40|2023-12-25|16      |IL |
|9       |36         |19900.76    |2023-12-25 10:31:40|2023-12-25|15      |IL |
|8       |35         |19900.76    |2023-

### Create Iceberg Tables

In [9]:
spark.sql("use  iceberg_catalog."+prod_db_nm)

DataFrame[]

# List the available tables.

In [10]:
spark.sql("show tables").show(100,False)


+---------+----------+-----------+
|namespace|tableName |isTemporary|
+---------+----------+-----------+
|db2      |order_h1  |false      |
|db2      |order_h2  |false      |
|db2      |customer_h|false      |
|db2      |orders    |false      |
|db2      |order_h   |false      |
+---------+----------+-----------+



In [11]:
spark.sql(f"""drop table if EXISTS iceberg_catalog."""+prod_db_nm+""".order_h """)

DataFrame[]

In [12]:
spark.sql(f"""CREATE TABLE IF NOT EXISTS iceberg_catalog."""+prod_db_nm+""".order_h ( 
order_id BIGINT, 
customer_id BIGINT, 
order_amount DECIMAL(10, 2), 
Order_ts  timestamp,
st string
)TBLPROPERTIES ( 
 'write.format.default'='parquet', 
 'write.delete.mode'='merge-on-read',
 'write.update.mode'='merge-on-read',
 'write.merge.mode'='merge-on-read', 
 'format-version' = '2')
PARTITIONED BY ( year(Order_ts), st) """)


DataFrame[]

In [13]:
spark.sql(f"""drop table if EXISTS iceberg_catalog."""+prod_db_nm+""".customer_h """)

DataFrame[]

In [14]:
spark.sql(f"""CREATE TABLE IF NOT EXISTS iceberg_catalog."""+prod_db_nm+""".customer_h ( 
customer_id BIGINT, 
name string, 
st  string
)TBLPROPERTIES ( 
 'write.format.default'='parquet', 
 'write.delete.mode'='merge-on-read',
 'write.update.mode'='merge-on-read',
 'write.merge.mode'='merge-on-read', 
 'format-version' = '2')
PARTITIONED BY ( st)  """)

DataFrame[]

# Check the metadata location of the Iceberg table.

In [15]:
!ls -l /tmp/ozone1/warehouse/db2/order_h/metadata


total 16
-rw-r--r--  1 satyak  wheel  1741 Jul 19 09:03 v1.metadata.json
-rw-r--r--  1 satyak  wheel     1 Jul 19 09:03 version-hint.text


In [16]:
spark.sql("insert into iceberg_catalog.db2.order_h values(1, 31, 19900.76,timestamp '2023-12-25T10:31:40', 'IL')")
spark.sql("insert into iceberg_catalog.db2.order_h values(2, 32, 19900.76,timestamp '2023-12-25T10:31:40', 'WI')")
spark.sql("insert into iceberg_catalog.db2.order_h values(3, 33, 19900.76,timestamp '2023-12-25T10:31:40', 'IL')")
spark.sql("insert into iceberg_catalog.db2.order_h values(4, 34, 19900.76,timestamp '2023-12-25T10:31:40', 'IL')")
spark.sql("insert into iceberg_catalog.db2.order_h values(5, 35, 19900.76,timestamp '2023-12-25T10:31:40', 'IL')")
spark.sql("insert into iceberg_catalog.db2.order_h values(6, 36, 19900.76,timestamp '2023-12-25T10:31:40', 'CA')")
spark.sql("insert into iceberg_catalog.db2.order_h values(7, 37, 19900.76,timestamp '2023-12-25T10:31:40', 'IL')")
spark.sql("insert into iceberg_catalog.db2.order_h values(8, 38, 19900.76,timestamp '2023-12-25T10:31:40', 'IL')")
spark.sql("insert into iceberg_catalog.db2.order_h values(9, 39, 19900.76,timestamp '2023-12-25T10:31:40', 'MI')")
spark.sql("insert into iceberg_catalog.db2.order_h values(10, 40, 19900.76,timestamp '2023-12-25T10:31:40', 'IL')")
spark.sql("insert into iceberg_catalog.db2.order_h values(11, 41, 19900.76,timestamp '2023-12-25T10:31:40', 'MI')")


DataFrame[]

In [17]:
spark.sql("insert into iceberg_catalog.db2.order_h values(11, 41, 19900.76,timestamp '2023-12-25T11:31:40', 'MI')")


DataFrame[]

In [18]:
spark.sql("insert into iceberg_catalog.db2.customer_h values(31, 'Bob31','IL')")
spark.sql("insert into iceberg_catalog.db2.customer_h values(32, 'Bob32','WI')")
spark.sql("insert into iceberg_catalog.db2.customer_h values(33, 'Bob33','IL')")
spark.sql("insert into iceberg_catalog.db2.customer_h values(34, 'Bob34','IL')")
spark.sql("insert into iceberg_catalog.db2.customer_h values(35, 'Bob35','IL')")
spark.sql("insert into iceberg_catalog.db2.customer_h values(36, 'Bob36','CA')")
spark.sql("insert into iceberg_catalog.db2.customer_h values(37, 'Bob37','IL')")
spark.sql("insert into iceberg_catalog.db2.customer_h values(38, 'Bob38','IL')")
spark.sql("insert into iceberg_catalog.db2.customer_h values(39, 'Bob39','MI')")
spark.sql("insert into iceberg_catalog.db2.customer_h values(40, 'Bob40','IL')")
spark.sql("insert into iceberg_catalog.db2.customer_h values(41, 'Bob41','MI')")


DataFrame[]

# Query the Iceberg table.

In [19]:

spark.sql("select * from iceberg_catalog."+prod_db_nm+".order_h ").show(100,False)


+--------+-----------+------------+-------------------+---+
|order_id|customer_id|order_amount|Order_ts           |st |
+--------+-----------+------------+-------------------+---+
|10      |40         |19900.76    |2023-12-25 10:31:40|IL |
|11      |41         |19900.76    |2023-12-25 11:31:40|MI |
|2       |32         |19900.76    |2023-12-25 10:31:40|WI |
|4       |34         |19900.76    |2023-12-25 10:31:40|IL |
|9       |39         |19900.76    |2023-12-25 10:31:40|MI |
|7       |37         |19900.76    |2023-12-25 10:31:40|IL |
|11      |41         |19900.76    |2023-12-25 10:31:40|MI |
|5       |35         |19900.76    |2023-12-25 10:31:40|IL |
|3       |33         |19900.76    |2023-12-25 10:31:40|IL |
|6       |36         |19900.76    |2023-12-25 10:31:40|CA |
|8       |38         |19900.76    |2023-12-25 10:31:40|IL |
|1       |31         |19900.76    |2023-12-25 10:31:40|IL |
+--------+-----------+------------+-------------------+---+



In [20]:

spark.sql("select * from iceberg_catalog."+prod_db_nm+".customer_h ").show(100,False)


+-----------+-----+---+
|customer_id|name |st |
+-----------+-----+---+
|39         |Bob39|MI |
|41         |Bob41|MI |
|31         |Bob31|IL |
|34         |Bob34|IL |
|33         |Bob33|IL |
|37         |Bob37|IL |
|35         |Bob35|IL |
|38         |Bob38|IL |
|36         |Bob36|CA |
|32         |Bob32|WI |
|40         |Bob40|IL |
+-----------+-----+---+



### Check the Iceberg table data files

In [21]:
!ls -l /tmp/ozone1/warehouse/db2/order_h/data/*/*


/tmp/ozone1/warehouse/db2/order_h/data/Order_ts_year=2023/st=CA:
total 8
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 00149-1030-c7897cc9-e78f-40f1-9f0e-8f5c176f22e5-0-00001.parquet

/tmp/ozone1/warehouse/db2/order_h/data/Order_ts_year=2023/st=IL:
total 56
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 00074-1231-3e32f5c1-b3a9-4d9b-a251-d6a07f13d1ce-0-00001.parquet
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 00074-1432-73c71ebf-6562-430a-ac26-f7d684fb48eb-0-00001.parquet
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 00074-1834-fc0c190b-f8d8-4614-891f-d980173f3f49-0-00001.parquet
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 00074-25-d0d8ac17-10cf-439f-a988-9028f2317920-0-00001.parquet
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 00074-427-41eb14c7-3e91-48b3-886d-ef2341e0cf3a-0-00001.parquet
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 00074-628-4a2be96b-9eab-40a8-ba8f-8a748073c901-0-00001.parquet
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 00074-829-43097a35-5dc3-4321-855

In [22]:
!ls -l /tmp/ozone1/warehouse/db2/customer_h/data/*

/tmp/ozone1/warehouse/db2/customer_h/data/st=CA:
total 8
-rw-r--r--  1 satyak  wheel  890 Jul 19 09:04 00065-3442-78e1dfad-2ffe-451e-8ebb-41e504ed2306-0-00001.parquet

/tmp/ozone1/warehouse/db2/customer_h/data/st=IL:
total 56
-rw-r--r--  1 satyak  wheel  890 Jul 19 09:04 00080-2437-1eb45764-1b0e-4ddb-9c65-6b270acd75c6-0-00001.parquet
-rw-r--r--  1 satyak  wheel  890 Jul 19 09:04 00080-2839-2e2a08cf-f321-4d40-8403-fd0d93534003-0-00001.parquet
-rw-r--r--  1 satyak  wheel  890 Jul 19 09:04 00080-3040-5bd60824-73c3-4ff5-833e-c3d23c913ee4-0-00001.parquet
-rw-r--r--  1 satyak  wheel  890 Jul 19 09:04 00080-3241-fe6c4310-0640-4f21-85c6-0f35281480cc-0-00001.parquet
-rw-r--r--  1 satyak  wheel  890 Jul 19 09:04 00080-3643-1405d1f4-4349-4abb-839a-bb89cf612671-0-00001.parquet
-rw-r--r--  1 satyak  wheel  890 Jul 19 09:04 00080-3844-77286856-bf08-4804-b1d1-724d61977fb9-0-00001.parquet
-rw-r--r--  1 satyak  wheel  890 Jul 19 09:04 00080-4246-ed2c6125-cc25-435b-985d-309656c40602-0-00001.parquet

/tm

# Hidden Partitoin Test

In [23]:
spark

# After every query run check spark UI for plan verification

### Query the entire table and 

In [24]:

spark.sql("select * from iceberg_catalog."+prod_db_nm+".order_h").show(100,False)


+--------+-----------+------------+-------------------+---+
|order_id|customer_id|order_amount|Order_ts           |st |
+--------+-----------+------------+-------------------+---+
|9       |39         |19900.76    |2023-12-25 10:31:40|MI |
|11      |41         |19900.76    |2023-12-25 11:31:40|MI |
|5       |35         |19900.76    |2023-12-25 10:31:40|IL |
|3       |33         |19900.76    |2023-12-25 10:31:40|IL |
|4       |34         |19900.76    |2023-12-25 10:31:40|IL |
|6       |36         |19900.76    |2023-12-25 10:31:40|CA |
|7       |37         |19900.76    |2023-12-25 10:31:40|IL |
|2       |32         |19900.76    |2023-12-25 10:31:40|WI |
|11      |41         |19900.76    |2023-12-25 10:31:40|MI |
|10      |40         |19900.76    |2023-12-25 10:31:40|IL |
|8       |38         |19900.76    |2023-12-25 10:31:40|IL |
|1       |31         |19900.76    |2023-12-25 10:31:40|IL |
+--------+-----------+------------+-------------------+---+



### Query one or two rows and check how many files are read. 

In [32]:
spark.sql("select * from iceberg_catalog."+prod_db_nm+".order_h where order_id=10").show(100,False)

+--------+-----------+------------+-------------------+---+
|order_id|customer_id|order_amount|Order_ts           |st |
+--------+-----------+------------+-------------------+---+
|10      |40         |19900.76    |2023-12-25 10:31:40|IL |
+--------+-----------+------------+-------------------+---+



In [26]:
spark.sql("select * from iceberg_catalog."+prod_db_nm+".order_h where order_id in(8,7)").show(100,False)

+--------+-----------+------------+-------------------+---+
|order_id|customer_id|order_amount|Order_ts           |st |
+--------+-----------+------------+-------------------+---+
|7       |37         |19900.76    |2023-12-25 10:31:40|IL |
|8       |38         |19900.76    |2023-12-25 10:31:40|IL |
+--------+-----------+------------+-------------------+---+



In [27]:

spark.sql("select * from iceberg_catalog."+prod_db_nm+".order_h").show(100,False)

+--------+-----------+------------+-------------------+---+
|order_id|customer_id|order_amount|Order_ts           |st |
+--------+-----------+------------+-------------------+---+
|3       |33         |19900.76    |2023-12-25 10:31:40|IL |
|2       |32         |19900.76    |2023-12-25 10:31:40|WI |
|5       |35         |19900.76    |2023-12-25 10:31:40|IL |
|10      |40         |19900.76    |2023-12-25 10:31:40|IL |
|11      |41         |19900.76    |2023-12-25 10:31:40|MI |
|6       |36         |19900.76    |2023-12-25 10:31:40|CA |
|4       |34         |19900.76    |2023-12-25 10:31:40|IL |
|9       |39         |19900.76    |2023-12-25 10:31:40|MI |
|7       |37         |19900.76    |2023-12-25 10:31:40|IL |
|11      |41         |19900.76    |2023-12-25 11:31:40|MI |
|8       |38         |19900.76    |2023-12-25 10:31:40|IL |
|1       |31         |19900.76    |2023-12-25 10:31:40|IL |
+--------+-----------+------------+-------------------+---+



In [28]:
spark.sql("select * from iceberg_catalog."+prod_db_nm+".order_h where customer_id=36").show(100,False)


+--------+-----------+------------+-------------------+---+
|order_id|customer_id|order_amount|Order_ts           |st |
+--------+-----------+------------+-------------------+---+
|6       |36         |19900.76    |2023-12-25 10:31:40|CA |
+--------+-----------+------------+-------------------+---+



# Execute Hive SQLs and Check Partition Pruning

For Hive, we need to add the partition key in an SQL 'WHERE' clause to take advantage of partition pruning.


In [29]:

spark.sql("select * from spark_catalog."+prod_db_nm+".order_hive").show(100,False)


+--------+-----------+------------+-------------------+----------+--------+---+
|order_id|customer_id|order_amount|Order_ts           |order_dt  |order_hr|st |
+--------+-----------+------------+-------------------+----------+--------+---+
|10      |37         |19900.76    |2023-12-25 10:31:40|2023-12-25|15      |IL |
|10      |37         |19900.76    |2023-12-25 10:31:40|2023-12-25|15      |IL |
|3       |31         |19900.76    |2023-12-25 10:31:40|2023-12-25|11      |IL |
|3       |31         |19900.76    |2023-12-25 10:31:40|2023-12-25|11      |IL |
|7       |34         |19900.76    |2023-12-25 10:31:40|2023-12-25|14      |IL |
|7       |34         |19900.76    |2023-12-25 10:31:40|2023-12-25|14      |IL |
|11      |38         |19900.76    |2023-12-25 10:31:40|2023-12-25|16      |IL |
|11      |38         |19900.76    |2023-12-25 10:31:40|2023-12-25|16      |IL |
|9       |36         |19900.76    |2023-12-25 10:31:40|2023-12-25|15      |IL |
|8       |35         |19900.76    |2023-

In [30]:
spark.sql("select * from spark_catalog."+prod_db_nm+".order_hive where order_id =10").show(100,False)


+--------+-----------+------------+-------------------+----------+--------+---+
|order_id|customer_id|order_amount|Order_ts           |order_dt  |order_hr|st |
+--------+-----------+------------+-------------------+----------+--------+---+
|10      |37         |19900.76    |2023-12-25 10:31:40|2023-12-25|15      |IL |
|10      |37         |19900.76    |2023-12-25 10:31:40|2023-12-25|15      |IL |
+--------+-----------+------------+-------------------+----------+--------+---+



In [31]:
spark.sql("""select * from spark_catalog."""+prod_db_nm+""".order_hive 
          where order_id =10 
          and order_dt='2023-12-25' 
          and order_hr=15 
          and st ='IL'  """).show(100,False)


+--------+-----------+------------+-------------------+----------+--------+---+
|order_id|customer_id|order_amount|Order_ts           |order_dt  |order_hr|st |
+--------+-----------+------------+-------------------+----------+--------+---+
|10      |37         |19900.76    |2023-12-25 10:31:40|2023-12-25|15      |IL |
|10      |37         |19900.76    |2023-12-25 10:31:40|2023-12-25|15      |IL |
+--------+-----------+------------+-------------------+----------+--------+---+



## Iceberg Joins.

In [33]:
spark.sql("""select * 
from iceberg_catalog."""+prod_db_nm+""".order_h a,
iceberg_catalog."""+prod_db_nm+""".customer_h b
where a.customer_id =b.customer_id """).show(100,False)


+--------+-----------+------------+-------------------+---+-----------+-----+---+
|order_id|customer_id|order_amount|Order_ts           |st |customer_id|name |st |
+--------+-----------+------------+-------------------+---+-----------+-----+---+
|7       |37         |19900.76    |2023-12-25 10:31:40|IL |37         |Bob37|IL |
|11      |41         |19900.76    |2023-12-25 10:31:40|MI |41         |Bob41|MI |
|11      |41         |19900.76    |2023-12-25 11:31:40|MI |41         |Bob41|MI |
|2       |32         |19900.76    |2023-12-25 10:31:40|WI |32         |Bob32|WI |
|10      |40         |19900.76    |2023-12-25 10:31:40|IL |40         |Bob40|IL |
|6       |36         |19900.76    |2023-12-25 10:31:40|CA |36         |Bob36|CA |
|4       |34         |19900.76    |2023-12-25 10:31:40|IL |34         |Bob34|IL |
|3       |33         |19900.76    |2023-12-25 10:31:40|IL |33         |Bob33|IL |
|9       |39         |19900.76    |2023-12-25 10:31:40|MI |39         |Bob39|MI |
|5       |35    

### Order table Partition Pruneing 

In [34]:
spark.sql("""select * 
from iceberg_catalog."""+prod_db_nm+""".order_h a,
iceberg_catalog."""+prod_db_nm+""".customer_h b
where a.customer_id =b.customer_id 
and a.Order_ts='2023-12-25T10:31:40'
and a.st= 'MI' """).show(100,False)

+--------+-----------+------------+-------------------+---+-----------+-----+---+
|order_id|customer_id|order_amount|Order_ts           |st |customer_id|name |st |
+--------+-----------+------------+-------------------+---+-----------+-----+---+
|11      |41         |19900.76    |2023-12-25 10:31:40|MI |41         |Bob41|MI |
|9       |39         |19900.76    |2023-12-25 10:31:40|MI |39         |Bob39|MI |
+--------+-----------+------------+-------------------+---+-----------+-----+---+



### customer table partition prune

In [35]:
spark.sql("""select * 
from iceberg_catalog."""+prod_db_nm+""".order_h a,
iceberg_catalog."""+prod_db_nm+""".customer_h b
where a.customer_id =b.customer_id 
and b.customer_id=31""").show(100,False)

+--------+-----------+------------+-------------------+---+-----------+-----+---+
|order_id|customer_id|order_amount|Order_ts           |st |customer_id|name |st |
+--------+-----------+------------+-------------------+---+-----------+-----+---+
|1       |31         |19900.76    |2023-12-25 10:31:40|IL |31         |Bob31|IL |
+--------+-----------+------------+-------------------+---+-----------+-----+---+



# Alter Partition key

In [36]:
spark.sql("ALTER TABLE iceberg_catalog."+prod_db_nm+".order_h ADD PARTITION field month(Order_ts)").show(100,False)

++
||
++
++



ANTLR Tool version 4.9.3 used for code generation does not match the current runtime version 4.8ANTLR Runtime version 4.9.3 used for parser compilation does not match the current runtime version 4.8ANTLR Tool version 4.9.3 used for code generation does not match the current runtime version 4.8ANTLR Runtime version 4.9.3 used for parser compilation does not match the current runtime version 4.8

In [37]:
spark.sql("desc extended   iceberg_catalog."+prod_db_nm+".order_h ").show(100,False)

+----------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                                                                                                                                                                         |comment|
+----------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|order_id                    |bigint                                                                                                                      

In [38]:
!ls -l /tmp/ozone1/warehouse/db2/order_h/data/*/*

/tmp/ozone1/warehouse/db2/order_h/data/Order_ts_year=2023/st=CA:
total 8
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 00149-1030-c7897cc9-e78f-40f1-9f0e-8f5c176f22e5-0-00001.parquet

/tmp/ozone1/warehouse/db2/order_h/data/Order_ts_year=2023/st=IL:
total 56
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 00074-1231-3e32f5c1-b3a9-4d9b-a251-d6a07f13d1ce-0-00001.parquet
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 00074-1432-73c71ebf-6562-430a-ac26-f7d684fb48eb-0-00001.parquet
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 00074-1834-fc0c190b-f8d8-4614-891f-d980173f3f49-0-00001.parquet
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 00074-25-d0d8ac17-10cf-439f-a988-9028f2317920-0-00001.parquet
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 00074-427-41eb14c7-3e91-48b3-886d-ef2341e0cf3a-0-00001.parquet
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 00074-628-4a2be96b-9eab-40a8-ba8f-8a748073c901-0-00001.parquet
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 00074-829-43097a35-5dc3-4321-855

In [39]:
spark.sql("insert into iceberg_catalog.db2.order_h values(1112, 4112, 19900.76,timestamp '2023-11-25T11:31:40', 'MI')")


DataFrame[]

In [40]:
spark.sql("insert into iceberg_catalog.db2.customer_h values(4112, 'Bob411','MI')")


DataFrame[]

In [41]:
spark.sql("insert into iceberg_catalog.db2.order_h values(8181, 99, 19900.76,timestamp '2024-06-25T11:31:40', 'MI')")


DataFrame[]

### Check file directory

In [42]:
!ls -l /tmp/ozone1/warehouse/db2/order_h/data/*/*/*

-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 /tmp/ozone1/warehouse/db2/order_h/data/Order_ts_year=2023/st=CA/00149-1030-c7897cc9-e78f-40f1-9f0e-8f5c176f22e5-0-00001.parquet
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 /tmp/ozone1/warehouse/db2/order_h/data/Order_ts_year=2023/st=IL/00074-1231-3e32f5c1-b3a9-4d9b-a251-d6a07f13d1ce-0-00001.parquet
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 /tmp/ozone1/warehouse/db2/order_h/data/Order_ts_year=2023/st=IL/00074-1432-73c71ebf-6562-430a-ac26-f7d684fb48eb-0-00001.parquet
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 /tmp/ozone1/warehouse/db2/order_h/data/Order_ts_year=2023/st=IL/00074-1834-fc0c190b-f8d8-4614-891f-d980173f3f49-0-00001.parquet
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 /tmp/ozone1/warehouse/db2/order_h/data/Order_ts_year=2023/st=IL/00074-25-d0d8ac17-10cf-439f-a988-9028f2317920-0-00001.parquet
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 /tmp/ozone1/warehouse/db2/order_h/data/Order_ts_year=2023/st=IL/00074-427-41eb14

In [43]:
spark.sql("""select * ,month(Order_ts)
from iceberg_catalog."""+prod_db_nm+""".order_h a
where order_id =8181 """).show(100,False)

+--------+-----------+------------+-------------------+---+---------------+
|order_id|customer_id|order_amount|Order_ts           |st |month(Order_ts)|
+--------+-----------+------------+-------------------+---+---------------+
|8181    |99         |19900.76    |2024-06-25 11:31:40|MI |6              |
+--------+-----------+------------+-------------------+---+---------------+



In [44]:
spark.sql("""select * ,month(Order_ts)
from iceberg_catalog."""+prod_db_nm+""".order_h a
where Order_ts between '2024-05-25T00:00:00' and '2024-07-25T00:00:00' """).show(100,False)

+--------+-----------+------------+-------------------+---+---------------+
|order_id|customer_id|order_amount|Order_ts           |st |month(Order_ts)|
+--------+-----------+------------+-------------------+---+---------------+
|8181    |99         |19900.76    |2024-06-25 11:31:40|MI |6              |
+--------+-----------+------------+-------------------+---+---------------+



In [45]:
spark.sql("""select * ,month(Order_ts)
from iceberg_catalog."""+prod_db_nm+""".order_h a
where Order_ts between '2024-06-25T11:00:00' and '2024-06-25T11:59:59' """).show(100,False)

+--------+-----------+------------+-------------------+---+---------------+
|order_id|customer_id|order_amount|Order_ts           |st |month(Order_ts)|
+--------+-----------+------------+-------------------+---+---------------+
|8181    |99         |19900.76    |2024-06-25 11:31:40|MI |6              |
+--------+-----------+------------+-------------------+---+---------------+



### Re write data files

In [46]:
spark.sql("CALL iceberg_catalog.system.rewrite_data_files(\
table => 'iceberg_catalog."+prod_db_nm+".order_h', \
options => map('target-file-size-bytes','40240000'))").show()

+--------------------------+----------------------+---------------------+
|rewritten_data_files_count|added_data_files_count|rewritten_bytes_count|
+--------------------------+----------------------+---------------------+
|                        12|                     4|                17711|
+--------------------------+----------------------+---------------------+



In [47]:
!ls -l /tmp/ozone1/warehouse/db2/order_h/data/*/*/*

-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 /tmp/ozone1/warehouse/db2/order_h/data/Order_ts_year=2023/st=CA/00149-1030-c7897cc9-e78f-40f1-9f0e-8f5c176f22e5-0-00001.parquet
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 /tmp/ozone1/warehouse/db2/order_h/data/Order_ts_year=2023/st=IL/00074-1231-3e32f5c1-b3a9-4d9b-a251-d6a07f13d1ce-0-00001.parquet
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 /tmp/ozone1/warehouse/db2/order_h/data/Order_ts_year=2023/st=IL/00074-1432-73c71ebf-6562-430a-ac26-f7d684fb48eb-0-00001.parquet
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 /tmp/ozone1/warehouse/db2/order_h/data/Order_ts_year=2023/st=IL/00074-1834-fc0c190b-f8d8-4614-891f-d980173f3f49-0-00001.parquet
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 /tmp/ozone1/warehouse/db2/order_h/data/Order_ts_year=2023/st=IL/00074-25-d0d8ac17-10cf-439f-a988-9028f2317920-0-00001.parquet
-rw-r--r--  1 satyak  wheel  1476 Jul 19 09:04 /tmp/ozone1/warehouse/db2/order_h/data/Order_ts_year=2023/st=IL/00074-427-41eb14